In [1080]:
import nfl_data_py as nfl
import pandas as pd
import numpy as np
from scipy.special.cython_special import nbdtr

In [1081]:
players = nfl.import_seasonal_rosters([2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2023,2024])
print(players.columns)
picks = nfl.import_draft_values()
print(picks)

Index(['season', 'team', 'position', 'depth_chart_position', 'jersey_number', 'status', 'player_name', 'first_name', 'last_name', 'birth_date', 'height', 'weight', 'college', 'player_id', 'espn_id', 'sportradar_id', 'yahoo_id', 'rotowire_id', 'pff_id', 'pfr_id', 'fantasy_data_id', 'sleeper_id', 'years_exp', 'headshot_url', 'ngs_position', 'week', 'game_type', 'status_description_abbr', 'football_name', 'esb_id', 'gsis_it_id', 'smart_id', 'entry_year', 'rookie_year', 'draft_club', 'draft_number', 'age'], dtype='object')
     pick  stuart  johnson     hill   otc   pff
0       1  34.600     3000 1000.000  3000 1.135
1       2  30.200     2600  717.170  2635 1.099
2       3  27.600     2200  514.330  2421 1.063
3       4  25.800     1800  490.520  2270 1.027
4       5  24.300     1700  467.810  2152 0.991
..    ...     ...      ...      ...   ...   ...
257   258   0.000        0    1.080    76 0.047
258   259   0.000        0    1.060    74 0.047
259   260   0.000        0    1.040    72 0

In [1082]:
players.rename(columns={'draft_number': 'pick'}, inplace=True)
players['pick'] = players['pick'].fillna(0).astype(int)
print(players)

       season team position depth_chart_position jersey_number status          player_name first_name   last_name birth_date  height  weight             college   player_id espn_id sportradar_id yahoo_id rotowire_id pff_id pfr_id fantasy_data_id sleeper_id  years_exp                                                                        headshot_url ngs_position  week game_type status_description_abbr football_name     esb_id gsis_it_id                              smart_id  entry_year  rookie_year draft_club  pick    age
0        2010  PIT        T                 None            71    ACT        Flozell Adams    Flozell       Adams 1975-05-18  79.000 338.000                None  00-0000045    None          None     None        None   None   None            None       None     12.000  https://static.www.nfl.com/image/private/f_auto,q_auto/league/tvzbhead7hjhqpcbilgc         None    21        SB                     A01       Flozell  ADA280969       None  32004144-4128-0969-9390-de3ae3

In [1083]:
player_draft_data = pd.merge(picks, players, on= 'pick', how='right')
print(player_draft_data)
print(player_draft_data.columns)

       pick  stuart  johnson    hill      otc   pff  season team position depth_chart_position jersey_number status          player_name first_name   last_name birth_date  height  weight             college   player_id espn_id sportradar_id yahoo_id rotowire_id pff_id pfr_id fantasy_data_id sleeper_id  years_exp                                                                        headshot_url ngs_position  week game_type status_description_abbr football_name     esb_id gsis_it_id                              smart_id  entry_year  rookie_year draft_club    age
0        38  11.400  520.000 157.490 1084.000 0.458    2010  PIT        T                 None            71    ACT        Flozell Adams    Flozell       Adams 1975-05-18  79.000 338.000                None  00-0000045    None          None     None        None   None   None            None       None     12.000  https://static.www.nfl.com/image/private/f_auto,q_auto/league/tvzbhead7hjhqpcbilgc         None    21        SB      

In [1084]:
player_draft_data = player_draft_data.loc[:,['season', 'team',
       'position', 'player_name', 'player_id', 'years_exp', 'entry_year', 'rookie_year', 'draft_club',
       'age', 'pick','pff', 'stuart', 'johnson', 'hill', 'otc']]

print(player_draft_data[player_draft_data['player_name'] == 'Joe Burrow'])
print(player_draft_data)

       season team position player_name   player_id  years_exp  entry_year  rookie_year draft_club    age  pick   pff  stuart  johnson     hill      otc
28292    2020  CIN       QB  Joe Burrow  00-0036442      0.000    2020.000     2020.000        CIN 23.000     1 1.135  34.600 3000.000 1000.000 3000.000
30763    2021  CIN       QB  Joe Burrow  00-0036442      1.000    2020.000     2020.000        CIN 24.000     1 1.135  34.600 3000.000 1000.000 3000.000
32816    2023  CIN       QB  Joe Burrow  00-0036442      3.000    2020.000     2020.000        CIN 26.000     1 1.135  34.600 3000.000 1000.000 3000.000
35604    2024  CIN       QB  Joe Burrow  00-0036442      4.000    2020.000     2020.000        CIN 27.000     1 1.135  34.600 3000.000 1000.000 3000.000
       season team position          player_name   player_id  years_exp  entry_year  rookie_year draft_club    age  pick   pff  stuart  johnson    hill      otc
0        2010  PIT        T        Flozell Adams  00-0000045     12.000   

In [1085]:
print(player_draft_data['position'].unique())

['T' 'K' 'LS' 'ILB' 'CB' 'QB' 'P' 'C' 'OLB' 'TE' 'DE' 'FS' 'DT' 'WR' 'G'
 'RB' 'NT' 'SS' 'FB' 'MLB' 'DB' 'LB' 'PR' 'KR' 'S' 'DL' 'OL' None]


In [1086]:
LB_positions = ['ILB', 'OLB', 'MLB', 'LB']
CB_positions = ['CB', 'FS', 'SS', 'DB']
DL_positions = ['DE', 'DT', 'NT']
OL_positions = ['C', 'G', 'T']
RB_positions = ['RB', 'FB']
DROP_positions = ['LS', 'PR', 'P', 'K']

player_draft_data['position'] = player_draft_data['position'].replace(LB_positions, 'LB')
player_draft_data['position'] = player_draft_data['position'].replace(CB_positions, 'CB')
player_draft_data['position'] = player_draft_data['position'].replace(DL_positions, 'DL')
player_draft_data['position'] = player_draft_data['position'].replace(OL_positions, 'OL')
player_draft_data['position'] = player_draft_data['position'].replace(RB_positions, 'RB')

player_draft_data = player_draft_data[~player_draft_data['position'].isin(DROP_positions)]

In [1087]:
print(player_draft_data['team'].unique())

['PIT' 'BLT' 'DEN' 'TB' 'CLV' 'JAX' 'DAL' 'NYJ' 'CHI' 'TEN' 'WAS' 'GB'
 'ARZ' 'NE' 'MIN' 'ATL' 'MIA' 'SEA' 'CIN' 'IND' 'NO' 'HST' 'SF' 'KC' 'BUF'
 'SD' 'SL' 'NYG' 'OAK' 'DET' 'PHI' 'CAR' 'BAL' 'CLE' 'ARI' 'LA' 'HOU'
 'LAC' 'LV']


In [1088]:
team_mapping = {
    'ARI': 'Cardinals',
    'ARZ': 'Cardinals',
    'ATL': 'Falcons',
    'BLT': 'Ravens',
    'BAL': 'Ravens',
    'BUF': 'Bills',
    'CAR': 'Panthers',
    'CHI': 'Bears',
    'CIN': 'Bengals',
    'CLV': 'Browns',
    'CLE': 'Browns',
    'DAL': 'Cowboys',
    'DEN': 'Broncos',
    'DET': 'Lions',
    'GB': 'Packers',
    'HST': 'Texans',
    'HOU': 'Texans',
    'IND': 'Colts',
    'JAX': 'Jaguars',
    'KC': 'Chiefs',
    'LAC': 'Chargers',
    'SD': 'Chargers',
    'LA': 'Rams',
    'SL': 'Rams',
    'MIA': 'Dolphins',
    'MIN': 'Vikings',
    'NE': 'Patriots',
    'NO': 'Saints',
    'NYG': 'Giants',
    'NYJ': 'Jets',
    'PHI': 'Eagles',
    'PIT': 'Steelers',
    'STL': 'Steelers',
    'SF': '49ers',
    'SEA': 'Seahawks',
    'TB': 'Buccaneers',
    'TEN': 'Titans',
    'WAS': 'Commanders',
    'OAK': 'Raiders',
    'LV': 'Raiders'
}

player_draft_data['draft_club'] = player_draft_data['draft_club'].replace(team_mapping)
player_draft_data['team'] = player_draft_data['team'].replace(team_mapping)


In [1089]:
player_draft_data['draft_value'] = (player_draft_data['pff'] / 1.135 + 
                                    player_draft_data['stuart'] / 34.6 + 
                                    player_draft_data['johnson'] / 3000 + 
                                    player_draft_data['hill'] / 1000 +
                                    player_draft_data['otc'] / 3000).fillna(0)


# Scale the max value to 100
max_value = player_draft_data['draft_value'].max()
player_draft_data['draft_value'] = (player_draft_data['draft_value'] / max_value) * 1000 if max_value > 0 else 0

slow_decay_rate = 0.1 # Slow decay rate for the first 4 years 
fast_decay_rate = 0.3 # Faster decay rate after 4 years # Apply the piecewise decay function 
def adjusted_value(years_exp, draft_value): 
    if years_exp <= 4: 
        return draft_value * np.exp(-slow_decay_rate * years_exp) 
    else: 
        return draft_value * np.exp(-fast_decay_rate * (years_exp - 4)) * np.exp(-slow_decay_rate * 4) 
player_draft_data['adjusted_value'] = player_draft_data.apply(lambda row: adjusted_value(row['years_exp'], row['draft_value']), axis=1) # Display the results print(player_draft_data)


In [1090]:
player_draft_data = player_draft_data.sort_values(by=['adjusted_value'], ascending=False)

# Set options to display more rows and columns
pd.set_option('display.max_rows', 20)  # Adjust the number as needed
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)  # Increase width for better readability
pd.set_option('display.float_format', '{:.3f}'.format)  # Format floats for readability
player_draft_data = player_draft_data.reset_index(drop=True)
player_draft_data.drop('draft_value', axis=1, inplace=True)

print(player_draft_data[player_draft_data['position'] == 'QB'].head(20))

    season        team position       player_name   player_id  years_exp  entry_year  rookie_year  draft_club    age  pick   pff  stuart  johnson     hill      otc  adjusted_value
0     2023    Panthers       QB       Bryce Young  00-0039150      0.000    2023.000     2023.000    Panthers 22.000     1 1.135  34.600 3000.000 1000.000 3000.000        1000.000
1     2015  Buccaneers       QB    Jameis Winston  00-0031503      0.000    2015.000     2015.000  Buccaneers 21.000     1 1.135  34.600 3000.000 1000.000 3000.000        1000.000
2     2021     Jaguars       QB   Trevor Lawrence  00-0036971      0.000    2021.000     2021.000     Jaguars 21.000     1 1.135  34.600 3000.000 1000.000 3000.000        1000.000
3     2018      Browns       QB    Baker Mayfield  00-0034855      0.000    2018.000     2018.000      Browns 23.000     1 1.135  34.600 3000.000 1000.000 3000.000        1000.000
4     2010        Rams       QB      Sam Bradford  00-0027854      0.000    2010.000     2010.000   

In [1091]:
grouped_by_data = player_draft_data.groupby(['season', 'draft_club', 'position'], as_index= False).sum()

grouped_by_data = grouped_by_data.loc[:,['season', 'draft_club',
       'position', 'adjusted_value']]
print(grouped_by_data)
grouped_by_data = grouped_by_data.sort_values(by=['season'], ascending=False)

print(grouped_by_data[grouped_by_data['season'] == 2019].head(10))


      season draft_club position  adjusted_value
0       2010      49ers       CB         636.536
1       2010      49ers       DL         514.197
2       2010      49ers       LB        1019.949
3       2010      49ers       OL        1837.250
4       2010      49ers       QB         496.585
...      ...        ...      ...             ...
3573    2024    Vikings       OL        1116.102
3574    2024    Vikings       QB         622.128
3575    2024    Vikings       RB         288.767
3576    2024    Vikings       TE         204.217
3577    2024    Vikings       WR         789.157

[3578 rows x 4 columns]
      season  draft_club position  adjusted_value
2386    2019       Colts       TE           0.000
2381    2019       Colts       DL         600.197
2382    2019       Colts       LB         664.032
2383    2019       Colts       OL        1481.103
2384    2019       Colts       QB           0.000
2385    2019       Colts       RB         205.117
2389    2019  Commanders       DL    

In [1092]:
grouped_by_data[(grouped_by_data['position'] == 'WR') & (grouped_by_data['draft_club'] == '49ers')]


,season,draft_club,position,adjusted_value
3329,2024,49ers,WR,1013.462
3073,2023,49ers,WR,871.596
2817,2021,49ers,WR,972.352
2562,2020,49ers,WR,1074.615
2307,2019,49ers,WR,544.023
2051,2018,49ers,WR,397.357
1796,2017,49ers,WR,154.603
1542,2016,49ers,WR,249.236
1282,2015,49ers,WR,392.803
1026,2014,49ers,WR,743.637


In [1103]:
player_draft_data = player_draft_data.sort_values(by=['season', 'team', 'position'], ascending=True)
pd.set_option('display.max_colwidth', None)
player_draft_data = player_draft_data.reset_index(drop=True)
pd.set_option('display.max_rows', 100000)
print(player_draft_data.tail(500))

       season      team position               player_name   player_id  years_exp  entry_year  rookie_year  draft_club    age  pick   pff  stuart  johnson    hill      otc  adjusted_value
35247    2024    Saints       WR              Dante Pettis  00-0034860      6.000    2018.000     2018.000       49ers 28.000    44 0.415  10.500  460.000 134.580 1007.000          95.111
35248    2024    Saints       WR                 Bub Means  00-0039386      0.000    2024.000     2024.000      Saints 23.000   170 0.098   1.900   23.000   7.590  295.000          50.969
35249    2024    Saints       WR  Marquez Valdes-Scantling  00-0034272      6.000    2018.000     2018.000     Packers 29.000   174 0.094   1.700   22.000   6.940  283.000          17.699
35250    2024    Saints       WR     Equanimeous St. Brown  00-0034279      6.000    2018.000     2018.000     Packers 27.000   207 0.063   0.600    9.000   3.330  192.000          10.534
35251    2024    Saints       WR        Cedrick Wilson Jr.  